In [1]:
import zipfile as zf
import sqlalchemy as sq
import kaggle as k
import pandas as pd 

In [2]:
# Extracting out the orders dataset from the kaggle using kaggle API
!kaggle datasets download gabrielsantello/wholesale-and-retail-orders-dataset -f orders.csv


Dataset URL: https://www.kaggle.com/datasets/gabrielsantello/wholesale-and-retail-orders-dataset
License(s): other




  0%|          | 0.00/3.02M [00:00<?, ?B/s]
 33%|███▎      | 1.00M/3.02M [00:01<00:03, 692kB/s]
 66%|██████▌   | 2.00M/3.02M [00:01<00:00, 1.31MB/s]
 99%|█████████▉| 3.00M/3.02M [00:02<00:00, 1.93MB/s]
100%|██████████| 3.02M/3.02M [00:02<00:00, 1.54MB/s]


In [3]:
# Extracting out the product-supplier dataset from kaggle
!kaggle datasets download gabrielsantello/wholesale-and-retail-orders-dataset -f product-supplier.csv

Dataset URL: https://www.kaggle.com/datasets/gabrielsantello/wholesale-and-retail-orders-dataset


  0%|          | 0.00/559k [00:00<?, ?B/s]
100%|██████████| 559k/559k [00:01<00:00, 326kB/s]
100%|██████████| 559k/559k [00:01<00:00, 326kB/s]



License(s): other



In [4]:
# Unzip the orders.csv.zip file to get the order.csv
zip_loc = zf.ZipFile('orders.csv.zip')
zip_loc.extractall() 
zip_loc.close()

In [5]:
orders = pd.read_csv('C:/Users/zeus/Desktop/Git Repositories/Kaggle-Pipeline/orders.csv')
product = pd.read_csv('C:/Users/zeus/Desktop/Git Repositories/Kaggle-Pipeline/product-supplier.csv')

In [6]:
orders.head()

,Customer ID,Customer Status,Date Order was placed,Delivery Date,Order ID,Product ID,Quantity Ordered,Total Retail Price for This Order,Cost Price Per Unit
0,579,Silver,01-Jan-17,07-Jan-17,123002578,220101400106,2,92.6,20.70
1,7574,SILVER,01-Jan-17,05-Jan-17,123004074,210201000009,1,21.7,9.95
2,28861,Gold,01-Jan-17,04-Jan-17,123000871,230100500068,1,1.7,0.80
3,43796,Gold,01-Jan-17,06-Jan-17,123002851,220100100633,1,47.9,24.05
4,54673,Gold,01-Jan-17,04-Jan-17,123003607,220200200043,1,36.9,18.30


In [7]:
product.head()

,Product ID,Product Line,Product Category,Product Group,Product Name,Supplier Country,Supplier Name,Supplier ID
0,210100100001,Children,Children Outdoors,"Outdoor things, Kids",Boy's and Girl's Ski Pants with Braces,NO,Scandinavian Clothing A/S,50
1,210100100002,Children,Children Outdoors,"Outdoor things, Kids",Children's Jacket,ES,Luna sastreria S.A.,4742
2,210100100003,Children,Children Outdoors,"Outdoor things, Kids",Children's Jacket Sidney,NO,Scandinavian Clothing A/S,50
3,210100100004,Children,Children Outdoors,"Outdoor things, Kids",Children's Rain Set,NO,Scandinavian Clothing A/S,50
4,210100100005,Children,Children Outdoors,"Outdoor things, Kids",Children's Rain Suit,NO,Scandinavian Clothing A/S,50
